In [2]:
!pip install --upgrade tensorflow

Requirement already up-to-date: tensorflow in /home/nbuser/anaconda3_501/lib/python3.6/site-packages (1.11.0)
You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
from PIL import Image
import numpy as np
import pandas as pd
import tensorflow as tf

In [6]:
rectSize = 5
inputImagePath = "post_disaster"
inputImageFile = "humblePost.tiff"
inputImage = Image.open(inputImagePath + '/' + inputImageFile)
inputImage = inputImage.resize((512,512),Image.ANTIALIAS)
ipx, ipy = inputImage.size
print(ipx,ipy)
outputImagePath = "classify_output"
outputImageFile = "humblePostRoad.tiff"
outputImage = inputImage.crop((rectSize//2, rectSize//2, ipx - (rectSize//2), ipy - (rectSize//2)))
opx, opy = outputImage.size


512 512


In [7]:
numeric_columns = []
for i in range(1,76):
    numeric_columns.append("c"+str(i))
#print(numeric_columns)
numeric_features = [ tf.feature_column.numeric_column(key = column)
                    for column in numeric_columns]
#print(numeric_features[0])
trainingSteps = 1000
total = 5000
classifierConfig = tf.estimator.RunConfig(save_checkpoints_secs = None, save_checkpoints_steps = trainingSteps)
hiddenUnits = [100, 150, 100, 50]
classes = 2
modelDir = 'model'

classifier = tf.estimator.DNNClassifier(feature_columns = numeric_features,
                                             hidden_units = hiddenUnits,
                                             n_classes = classes,
                                             model_dir = modelDir,
                                             config = classifierConfig)

INFO:tensorflow:Using config: {'_model_dir': 'model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fb0a4dc7b00>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [8]:
def extractFeatures():
    features = np.zeros((((ipx - ((rectSize//2)*2)) * (ipy - ((rectSize//2)*2))), rectSize*rectSize*3), dtype=np.int)
    print(features.shape)
    rowIndex = 0
    
    for x in range(rectSize//2, ipx -(rectSize//2) ):
        for y in range(rectSize//2, ipy -(rectSize//2) ):
            rect = (x - (rectSize//2), y - (rectSize//2), x + (rectSize//2) + 1,y + (rectSize//2) + 1 )
            subImage = inputImage.crop(rect).load()
            colIndex = 0       
            for i in range(rectSize):
                for j in range(rectSize):
                    features[rowIndex, colIndex] = subImage[i, j][0]
                    colIndex += 1
                    features[rowIndex, colIndex] = subImage[i, j][1]
                    colIndex += 1
                    features[rowIndex, colIndex] = subImage[i, j][2]
                    colIndex += 1
            rowIndex += 1
    return features

def constructOutputImage(predictions):
    outputImgPixels = outputImage.load()
    print(len(predictions))
    print(opx*opy)
    rowIndex = 0
    for x in range(opx):
        for y in range(opy):
            if(rowIndex >= len(predictions)):
                outputImgPixels[x, y] = (0, 0, 0)
            if(predictions[rowIndex]['class_ids'][0]):
                outputImgPixels[x, y] = (255, 255, 255)
            else:
                outputImgPixels[x, y] = (0, 0, 0)
            rowIndex += 1
    


# predictions = list(classifier.predict(input_fn = extractFeatures))
# print(predictions)
# constructOutputImage(predictions)
# outputImage.save(outputImagePath + '/' + outputImageFile, 'JPEG')


In [9]:
x = extractFeatures()

(258064, 75)


In [10]:
# y = x.T
col = []
for i in range(1,76):
    col.append("c"+str(i))
print(col)
# featureDict = {}
# for i in range(len(y)):
#     featureDict[col[i]] = list(y[i])
    
# print(featureDict)
print(x)
x_predict = pd.DataFrame(data = x, columns = col, index = None)
print(x_predict)
predict_input_fn = tf.estimator.inputs.pandas_input_fn(
                    x = x_predict,
                    y = None,
                    batch_size = 32,
                    shuffle = False,
                    num_epochs = 1    
                    )    

['c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9', 'c10', 'c11', 'c12', 'c13', 'c14', 'c15', 'c16', 'c17', 'c18', 'c19', 'c20', 'c21', 'c22', 'c23', 'c24', 'c25', 'c26', 'c27', 'c28', 'c29', 'c30', 'c31', 'c32', 'c33', 'c34', 'c35', 'c36', 'c37', 'c38', 'c39', 'c40', 'c41', 'c42', 'c43', 'c44', 'c45', 'c46', 'c47', 'c48', 'c49', 'c50', 'c51', 'c52', 'c53', 'c54', 'c55', 'c56', 'c57', 'c58', 'c59', 'c60', 'c61', 'c62', 'c63', 'c64', 'c65', 'c66', 'c67', 'c68', 'c69', 'c70', 'c71', 'c72', 'c73', 'c74', 'c75']
[[106  97 100 ...  51  65  66]
 [ 95  95  96 ...  67  78  82]
 [ 77  88  87 ...  69  74  78]
 ...
 [ 58  66  70 ...  37  45  51]
 [ 54  65  66 ...  38  46  53]
 [ 51  63  63 ...  27  35  41]]
         c1   c2   c3   c4   c5   c6   c7   c8   c9  c10 ...   c66  c67  c68  \
0       106   97  100   95   95   96   77   88   87   61 ...    85   57   68   
1        95   95   96   77   88   87   61   75   72   46 ...    67   53   66   
2        77   88   87   61   75   72   46   60   59

In [11]:
predictions = list(classifier.predict(input_fn = predict_input_fn))
print(predictions[0])
constructOutputImage(predictions)
outputImage.save(outputImagePath + '/' + outputImageFile, 'TIFF')


Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model/model.ckpt-4000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
{'logits': array([1.0063846], dtype=float32), 'logistic': array([0.732312], dtype=float32), 'probabilities': array([0.26768798, 0.732312  ], dtype=float32), 'class_ids': array([1]), 'classes': array([b'1'], dtype=object)}
258064
258064
